# TECHIN 510
## Milestone 2

**Name**: Saif Mustafa

**Email**: saifm@uw.edu

**Student Number**: 1428039

---

For the second milestone, you will design and implement the interactive behaviors of your robot as specified below for the project you picked.

Your robot's interactive behaviors can build on functionalities you have implemented in recent labs. For example, you can use the python server technique you developed in lab 5, the interactive robot face and speech recognition functionality you developed in Lab 6, and the sensor capturing functionalities you developed in Lab 7. 

The interaction should involve a dialog between the user and the robot, using a combination of input and output channels on the robot.

**Channels for user input include:**
- Speech
- Touch-screen
- Camera
- Physical movement of device (motion/orientation sensing)

**Channels for robot output include:**
- Robot speech or sounds
- Text or visualizations on screen
- Gaze and facial expressions

In addition, you will incorporate one of the visual recognition functionalities you developed in Milestone 1. You can do this by creating a python server using the code you create in Milestone 1 (similar to the server we created in Lab 5), developing visual processing functionality in JavaScript (similar to camera data processing in Lab 7) or using a web service that does visual processing (will be covered on Week 8). Preferably this functionality should be the same as one of the functionalities from Milestone 1, but you can also change your mind and pick a new functionality. 

To complete this milestone you will make a video that demonstrates your robot's capabilities. The video should include an end-to-end dialog between a person and the robot, while demonstrating the interactive behavior enabled by the visual processing capability (e.g. the robot pauses if the person disappears). Your video does not need to include narrative about the project, but should be edited to the essential parts the interaction (i.e., remove irrelevant parts of your recording). 

Complete this milestone by submitting a link to your video on Canvas by Dec 10, as well as any materials (code, data, physical prompts) that would allow us to recreate the demo shown in the video. Do not upload the video to Canvas; instead make sure it can be viewed on a browser through a link (Youtube, Vimeo, Dropbox, Google Drive, etcetera).

---

## Topic: Nike Exercise and Wellness Robot

**Problem:** Many people who know the importance and potential benefits of exercising and meditation have a hard time motivating themselves to actually do them.

**Proposed solution:** Social robots have been demonstrated to have the impact of a social accountability partner in committing to difficult behavioral changes. This project will explore this potential of social robots for exercise, yoga, and/or meditation motivation and guidance.

**Prototype specifications:** The robot will have one user. The robot should interact with the user to introduce itself, meet its user, set user goals, and motivate the user to reach those goals. It should also guide the user through a sample exercise.

**Image processing capabilities for this robot (Milestone 1):**
- determine when a person is in front of the robot, 
- recognize whether the person is the owner of the robot, 
- determine the mood of the person, 
- determine when a person has completed an exercise, 
- determine when a functionality activation card (e.g. to start a specific exercise) is visually shown to the robot.

**Interactive robot behaviors of this robot (Milestone 2):**

Basic behaviors:
- The robot walks the user through steps of an exercise routine, verbally explaining the different movements/actions. The robot moves onto the next movements/action upon verbal confirmation from the user, e.g. saying "next move".

Behaviors that depend on image processing capabilities:
- Robot automatically stops when the person is not detected
- Robot acknowledges when the person has successfully performed the exercise and automatically moves on
- Robot automatically moves to an exercise requested by the user through a card shown to the robot

---

##  Installs / Imports

In [1]:
# installing opencv and mediapipe https://google.github.io/mediapipe/
# !pip install mediapipe
# !pip install opencv-python
# !pip install tensorflow
# !pip install deepface 
# !pip install tflearn

import mediapipe as mp
import cv2
import uuid
import time
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import os
import json

# speech stuff
import speech_recognition as sr
import pyttsx3
import webbrowser
import playsound
from gtts import gTTS
import random

mp_drawing = mp.solutions.drawing_utils # drawing model
mp_pose = mp.solutions.pose # pose estimation model
mp_hands = mp.solutions.hands # hand detection model (20 dots)

Instructions for updating:
non-resource variables are not supported in the long term


## Define training set + folders

In [2]:
emotions = ['happy','sad','angry','surprised','neutral']

In [3]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        
        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(0)
    
    
    for emotion in emotions:
        
        img_id = 0
        time.sleep(5)
        
        while True:
            ret, frame = cap.read()
            if face_cropped(frame) is not None:
                img_id +=1
                face = cv2.resize(face_cropped(frame), (200,200))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = "data/"+emotion+"."+str(img_id)+".jpg"
                cv2.imwrite(file_name_path, face)
                cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_DUPLEX, 
                            1, (0,255,0), 2)
                cv2.imshow("Cropped", face)
                if cv2.waitKey(1) == 13 or int(img_id)==250 :
                    break
        
    cap.release()
    cv2.destroyAllWindows()
    print("All image data collected.")

<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-3-bd93f35f0307>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [4]:
# generate_dataset()

## Label captured images

In [5]:
def my_label(image_name):
    emotion = image_name.split('.')[-3]
    
    #print("Emotion =",emotion)
    
    if emotion == 'happy':
        return np.array([1,0,0,0,0])
    elif emotion == 'sad':
        return np.array([0,1,0,0,0])
    elif emotion == 'angry':
        return np.array([0,0,1,0,0])
    elif emotion == 'surprised':
        return np.array([0,0,0,1,0])
    else:
        return np.array([0,0,0,0,1])

## Create model data using captured images

In [6]:
from random import shuffle
from tqdm import tqdm

def my_data():
    data = []
    for img in tqdm(os.listdir("data")):
        path = os.path.join("data",img)
        img_data=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data=cv2.resize(img_data, (50,50))
        data.append([np.array(img_data), my_label(img)])
    shuffle(data)
    return data

In [7]:
data = my_data() # remove DS.STORE file from github and rerun

100%|██████████| 1250/1250 [00:00<00:00, 1280.05it/s]


## Train Test Split

In [8]:
train = data[:875]
test = data[875:1250]

X_train, y_train, X_test, y_test = (np.array([i[0] for i in train]).reshape(-1,50,50,1), 
                                    [i[1] for i in train],
                                    np.array([i[0] for i in test]).reshape(-1,50,50,1),
                                    [i[1] for i in test])

In [9]:
# tf.reset_default_graph() # deprecated function
# https://stackoverflow.com/questions/40782271/attributeerror-module-tensorflow-has-no-attribute-reset-default-graph

tf.compat.v1.reset_default_graph()

convnet = input_data(shape=[50,50,1])
convnet = conv_2d(convnet,32,5,activation='relu') # 32 filters with stride = 5
convnet = max_pool_2d(convnet,5)
convnet = conv_2d(convnet,64,5,activation='relu')
convnet = max_pool_2d(convnet,5)
convnet = conv_2d(convnet,128,5,activation='relu')
convnet = max_pool_2d(convnet,5)
convnet = conv_2d(convnet,64,5,activation='relu')
convnet = max_pool_2d(convnet,5)
convnet = conv_2d(convnet,32,5,activation='relu')
convnet = max_pool_2d(convnet,5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 5, activation='softmax') # 5 emotions for output layer
convnet = regression(convnet, optimizer='adam', learning_rate = 0.001, loss='categorical_crossentropy')

model = tflearn.DNN(convnet, tensorboard_verbose=3)

model.fit(X_train, 
          y_train,
          n_epoch=20, 
          validation_set=(X_test, y_test),
          show_metric=True,
          run_id="emotion_detector")

Training Step: 279  | total loss: 0.34446 | time: 4.668s
| Adam | epoch: 020 | loss: 0.34446 - acc: 0.9497 -- iter: 832/875
Training Step: 280  | total loss: 0.31268 | time: 5.971s
| Adam | epoch: 020 | loss: 0.31268 - acc: 0.9547 | val_loss: 0.19034 - val_acc: 0.9440 -- iter: 875/875
--


In [10]:
# mp_drawing.draw_landmarks??

## Identifying joints

Mediapipe all usable body landmarks.

- 33 landmarks in total
- index starting at 0
- represent joints within the pose


![Mediapipe body landmarks](https://google.github.io/mediapipe/images/mobile/pose_tracking_full_body_landmarks.png)

## Angle calculations

In [11]:
# get angle between any 3 given points
def get_angle(p1, p2, p3):
    
    p1 = np.array(p1)
    p2 = np.array(p2)
    p3 = np.array(p3)
    
    radians = np.arctan2(p3[1]-p2[1], p3[0]-p2[0]) - np.arctan2(p1[1]-p2[1], p1[0]-p2[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

## Identify left and right hand

In [12]:
def get_label(index, hand, results):
    output = None
    for idx, classification in enumerate(results.multu_handedness):
        if classification.classification[0].index == index:
            
            label = classification.classification[0].label
            score = classification.classification[0].score
            text = '{} {}'.format(label, rount(score, 2))
            
            coords = tuple(np.multiply(np.array((hand.landmark[mp_hands.HandLandmark.WRIST].x,
                                                 hand.landmark[mp_hands.HandLandmark.WRIST].y)),
                                       [1280,720]).astype(int))
            
            output = text, coords
            
    return output

## Identify finger angles

![Mediapipe hand landmarks](https://google.github.io/mediapipe/images/mobile/hand_landmarks.png)

In [13]:
joint_list = [[8,6,5], # index
              [12,10,9], # middle
               [16,14,13],
               [20,18,17],
             [4,2,1]] # thumb

In [14]:
def draw_finger_angles(image, results, joint_list):  
    for hand in results.multi_hand_landmarks:
        for joint in joint_list:
            
            a = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y]) 
            b = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y]) 
            c = np.array([hand.landmark[joint[2]].x, hand.landmark[joint[2]].y]) 
            
            radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
            angle = np.abs(radians*100.0/np.pi)
            
#             if angle>180.0:
#                 angle=360.0-angle
                
            cv2.putText(image, 
                       str(round(angle,2)), tuple(np.multiply(b, [1280,720]).astype(int)),
                       cv2.FONT_HERSHEY_DUPLEX, 0.5, (255,255,255),2,cv2.LINE_AA)
            
            #print(angle)
            
    return image

In [15]:
def get_finger_angle(image, results, joint): 

    for hand in results.multi_hand_landmarks:
        a = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y]) 
        b = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y]) 
        c = np.array([hand.landmark[joint[2]].x, hand.landmark[joint[2]].y]) 
            
        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians*100.0/np.pi)

#             if angle>180.0:
#                 angle=360.0-angle

        cv2.putText(image, 
                   str(round(angle,2)), tuple(np.multiply(b, [1280,720]).astype(int)),
                   cv2.FONT_HERSHEY_DUPLEX, 0.5, (255,255,255),2,cv2.LINE_AA)

        #print(angle)

    return angle

## Voice input and output methods

In [16]:
def record_audio():
    with sr.Microphone() as source:
        audio = r.listen(source)
        voice_data = ''
        try:
            voice_data = r.recognize_google(audio)
            print(voice_data)
        except sr.UnknownValueError:
            print("Sorry, I didn't get that")
        except sr.RequestError:
            print("Sorry, speech service is down")
        return voice_data
            
def alexis_speak(audio_string):
    tts = gTTS(text=audio_string, lang='en')
    r = random.randint(1,10000000)
    audio_file = 'audio-' + str(r) + '.mp3'
    tts.save(audio_file)
    playsound.playsound(audio_file)
    print(audio_string)
    os.remove(audio_file)
        
def respond(voice_data):
    if 'your name' in voice_data:
        alexis_speak('My name is Nike')
        
    if 'calories' in voice_data:
        alexis_speak('You have completed X reps, which means you have burned (X/10)*40 calories')
        
    if 'search' in voice_data:
        url = 'https://assets.vogue.com/photos/5891e18123f9887c0e0e3f44/master/w_1600,c_limit/karlie-core-1.gif'
        webbrowser.get().open(url)
        #alexis_speak("Here's what I found")
    
    voice_data=''

## Building a counter / tracker

In [23]:
##### https://docs.opencv.org/master/dd/d43/tutorial_py_video_display.html

cap = cv2.VideoCapture(0)

faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

print("Saif")

if not cap.isOpened():
    print("Cannot open camera")
    exit()
    
curl_count=0
curl_flag_left="down"
curl_flag_right="False"
current_emotion = "neutral"
x=0
y=0
h=0
detected_emotion_index=0

# voice stuff
r = sr.Recognizer()
voice_data = ''

color_bg_ex1 = (255,255,255)
color_bg_ex2 = (255,255,255)


# alexis_speak("Today I'll be walking you through some exercises.")
# alexis_speak("To get started, please tell me what exercise you'd like")

# setting detection and tracking confidence    
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose, mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:

    while True:
        
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        #result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        faces = faceCascade.detectMultiScale(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),1.1,4)

        for(x,y,w,h) in faces:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0),2)
            gray_img = cv2.cvtColor(frame[y:y+h, x:x+w], cv2.COLOR_BGR2GRAY)
            gray_img = cv2.resize(gray_img,(50,50),interpolation=cv2.INTER_AREA)
            gray_img = gray_img.reshape(50,50,1) #  needs to be 50,50,1
            # print(gray_img.shape) needs to be 50,50,1
            detected_emotion_index = model.predict([gray_img])[0]
            # print(np.argmax(emotion_detection))
            # print(emotions[np.argmax(emotion_detection)])
            x=x
            y=y
            h=h
        
        # if frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        # Our operations on the frame come here
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make pose detection
        results = pose.process(image)
        
        # make hand detection
        results2 = hands.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract all joints
        try:
#             voice_data = record_audio()
#             respond(voice_data)
            
            landmarks = results.pose_landmarks.landmark
            
            if results2.multi_hand_landmarks:
                #print("saw hands")
                for num, hand in enumerate(results2.multi_hand_landmarks):
                    mp_drawing.draw_landmarks(image,
                                              hand,
                                              mp_hands.HAND_CONNECTIONS,
                                              mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2), # dots
                                              mp_drawing.DrawingSpec(color=(255,255,0), thickness=2, circle_radius=2))
                    
#                     if get_label(num, hand, results):
#                         text, coord = get_label(num, hand, results)
#                         cv2.putText(image, text, coord, cv2.FONT_HERSHEY_DUPLEX, 1, (255,255,255), 2, cv2.LINE_AA)          
                
                #draw_finger_angles(image, results2, joint_list)
            
            if get_finger_angle(image, results2, joint_list[0]) >= 97 and get_finger_angle(image, results2, joint_list[0]) <= 102 and get_finger_angle(image, results2, joint_list[1]) < 97:
                color_bg_ex1 = (69,255,213)
                color_bg_ex2 = (255,255,255)
                curl_count = 0
            
            if get_finger_angle(image, results2, joint_list[1]) >= 97 and get_finger_angle(image, results2, joint_list[1]) <= 102 and get_finger_angle(image, results2, joint_list[0]) >=97:
                color_bg_ex2 = (69,255,213)
                color_bg_ex1 = (255,255,255)
                curl_count = 0
            
            # shoulder coordinates
            shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            #print("Shoulder left =",shoulder_left)
            
            shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            #print("Shoulder right =",shoulder_right)
            
            # elbow coordinates
            elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

            # wrist coordinates
            wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # calculate angle
            # logic:
            # - if angle is > 90, then it's a curl down
            # - if angle < 90, the it's a curl up
            # - some kind of counter that starts and countfs every 2 as 1 successful curl
            # - visualize on screen
            curl_angle_left = get_angle(shoulder_left, elbow_left, wrist_left)
            #print("Curl angle left =", curl_angle_left)
            curl_angle_right = get_angle(shoulder_right, elbow_right, wrist_right)
            #print("Curl angle right =", curl_angle_right)
            
            # show angle at elbow
            cv2.putText(image, 
                        str(round(curl_angle_right)), 
                        tuple(np.multiply(elbow_right, [1280,720]).astype(int)),
                        cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)

            cv2.putText(image, 
                        str(round(curl_angle_left)), 
                        tuple(np.multiply(elbow_left, [1280,720]).astype(int)),
                        cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)
            
            # print("Count = ",curl_count)
            
            if curl_angle_left > 160:
                curl_flag_left = False # curl down
            if curl_angle_left < 50 and not curl_flag_left:
                curl_flag_left = True # curl up
                curl_count+=1
                #print("Count inside = ", curl_count)
                
            if curl_angle_right > 160:
                curl_flag_right=False # curl down
            if curl_angle_right < 50 and not curl_flag_right:
                curl_flag_right=True # curl up
                curl_count+=1
                #print("Count inside = ", curl_count)
            
        except:
            pass
        
        # display count
        #cv2.rectangle(image, start_point, end_point, color, thickness)
        #cv2.rectangle(image,(0,620),(1280,720),(0,0,0),-1)
        
        ## EXERCISE 1 TEXT
        cv2.rectangle(image, (350,610), (560,710), color_bg_ex1, -1) # rgba(213,255,69,255)
        cv2.putText(image, "EXERCISE 1", (360,635),
                    cv2.FONT_HERSHEY_DUPLEX, .4, (120,116,124), 1, cv2.LINE_AA) # rgb(124,116,120)
        cv2.putText(image, "BICEP CURLS", (360,675),
                    cv2.FONT_HERSHEY_DUPLEX, .8, (0,0,0), 2, cv2.LINE_AA)
        
        ## EXERCISE 2 TEXT   
        cv2.rectangle(image, (740,610), (970,710), color_bg_ex2, -1) # rgba(213,255,69,255)
        cv2.putText(image, "EXERCISE 2", (750,635),
                    cv2.FONT_HERSHEY_DUPLEX, .4, (120,116,124), 1, cv2.LINE_AA) # rgb(124,116,120)
        cv2.putText(image, "PUSH UPS", (750,675),
                    cv2.FONT_HERSHEY_DUPLEX, .8, (0,0,0), 2, cv2.LINE_AA)
        
        
        cv2.putText(image, "NIKE FITNESS TRACKER", (500,600),
                    cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        cv2.rectangle(image, (600,610), (700,710), (69,255,213), -1) # rgba(213,255,69,255)
        cv2.putText(image, "REPS", (635,635),
                    cv2.FONT_HERSHEY_DUPLEX, .4, (120,116,124), 1, cv2.LINE_AA) # rgb(124,116,120)
        cv2.putText(image, str(curl_count), (640,675),
                    cv2.FONT_HERSHEY_DUPLEX, 1, (0,0,0), 2, cv2.LINE_AA) # rgb(124,116,120)
        
        ###
        
        completed = False
        
        if(curl_count==0):
            cv2.putText(image, 
                    "Let's see you do 30 rep curls!",
                    (570,30),
                    cv2.FONT_HERSHEY_DUPLEX, 0.75, (255,255,255), 2, cv2.LINE_AA)
            if completed:
                time.sleep(5)
                completed=False
        
        if(curl_count>0 and curl_count<30):
            cv2.putText(image, 
                    "Nice form, keep going! " + str(30-curl_count) + " more reps to go",
                    (500,30),
                    cv2.FONT_HERSHEY_DUPLEX, 0.75, (255,255,255), 2, cv2.LINE_AA)
            
        if(curl_count>=30):
            cv2.putText(image, 
                    "Well done! Take a break!",
                    (570,30),
                    cv2.FONT_HERSHEY_DUPLEX, 0.75, (255,255,255), 2, cv2.LINE_AA)
            
            curl_count=0
            completed=True
            color_bg_ex1 = (255,255,255)
            color_bg_ex2 = (255,255,255)
        
        cv2.putText(image, 
                    "Welcome Saif, M25",
                    (x,y),
                    cv2.FONT_HERSHEY_DUPLEX, 0.75, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.putText(image, 
                    "why u " + emotions[np.argmax(detected_emotion_index)],
                    (x+25,y+h+25),
                    cv2.FONT_HERSHEY_DUPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
        
        ###
        
        
        # basic detection using the draw_landmarks utility
        mp_drawing.draw_landmarks(image, # image
                                  results.pose_landmarks, # coordinates
                                  mp_pose.POSE_CONNECTIONS, # pose connections
                                  mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2), # dots
                                  mp_drawing.DrawingSpec(thickness=2, circle_radius=2)) # connections
        
        # Display the resulting frame
        cv2.imshow('NIKE WELLNESS DETECTOR', image)
        
        if cv2.waitKey(1) == ord('q'):
            break
    
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

Saif


KeyboardInterrupt: 